# Data Set Construction

In this notebook we contruct the database of all regular season NBA games from 2016-17 to 2018-19 season. We obtain the following variables:

- *Target*
 - Combined score (TOT_PTS)
- *Features* 
 - Number of days between games (DAYS_BTWN_GAMES_x, DAYS_BTWN_GAMES_y)
 - Number of wins in last 10 games (WINS_10GAMES_x, WINS_10GAMES_y)
 - Wins thus far in the season (WINS_UPTOGAME_x, WINS_UPTOGAME_y)
 - Combined points in previous matchup (pre_PTS)
 - Distance traveled (distance_miles)
 - Rivalry (rivalry)
 - Avg. points/team in last 2,3 and 4 games (AVGPOINTS_2GAMES_x,...AVGPOINTS_4GAMES_y)
 - Referees (not used in analysis)

In [12]:
import pandas as pd
import numpy as np
import requests
import nba_api
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import boxscoresummaryv2

In [13]:
# Get basic game info for 2016-2019 NBA seasons, regular seasons only

gamefinder_1 = leaguegamefinder.LeagueGameFinder(
    season_nullable='2016-17',
    season_type_nullable='Regular Season',
    league_id_nullable='00')
games_1 = gamefinder_1.get_data_frames()[0]

gamefinder_2 = leaguegamefinder.LeagueGameFinder(
    season_nullable='2017-18',
    season_type_nullable='Regular Season',
    league_id_nullable='00')
games_2 = gamefinder_2.get_data_frames()[0]

gamefinder_3 = leaguegamefinder.LeagueGameFinder(
    season_nullable='2018-19',
    season_type_nullable='Regular Season',
    league_id_nullable='00')
games_3 = gamefinder_3.get_data_frames()[0]

In [23]:
#Combine datasets so all three seasons are in one dataset
games = games_1.append([games_2,games_3])

In [24]:
#Remove variables we don't need & make game date into date format
games = games.drop(columns=['MIN','FGM','FGA','FG_PCT','FG3M','FG3A','FG3_PCT','FTM','FTA','FT_PCT','OREB','DREB','REB','AST','STL','BLK','TOV','PF','PLUS_MINUS'])
games['GAME_DATE'] = pd.to_datetime(games.GAME_DATE)

In [25]:
#Calculate whether there are back-to-back games

#Sort by team & game date
games = games.sort_values(by=['SEASON_ID','TEAM_ID','GAME_DATE'], ascending=True)

#Calculate number of days between games
#Note: the sort above must be done before this command or it will be wrong
games['DAYS_BTWN_GAMES'] = games.groupby(['SEASON_ID','TEAM_ID'])['GAME_DATE'].diff() / np.timedelta64(1, 'D') 
games['DAYS_BTWN_GAMES'] = games['DAYS_BTWN_GAMES'].fillna(0) 


In [26]:
games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,PTS,DAYS_BTWN_GAMES
2428,22016,1610612737,ATL,Atlanta Hawks,0021600014,2016-10-27,ATL vs. WAS,W,114,0.0
2395,22016,1610612737,ATL,Atlanta Hawks,0021600026,2016-10-29,ATL @ PHI,W,104,2.0
2375,22016,1610612737,ATL,Atlanta Hawks,0021600044,2016-10-31,ATL vs. SAC,W,106,2.0
2350,22016,1610612737,ATL,Atlanta Hawks,0021600059,2016-11-02,ATL vs. LAL,L,116,2.0
2319,22016,1610612737,ATL,Atlanta Hawks,0021600070,2016-11-04,ATL @ WAS,L,92,2.0


In [27]:
# Calculate the team's number of wins in the last 10 games

# Generate dummy variable from WL variable
dummy = pd.get_dummies(games['WL'])
games = pd.concat([games, dummy], axis=1)
games.head()

#Sort by team & game date
games = games.sort_values(by=['SEASON_ID','TEAM_ID','GAME_DATE'], ascending=True)

# Calculate number of wins over last ten games
f = lambda x: x.rolling(min_periods=1, window=10).sum().shift()

games1 = games.groupby(['SEASON_ID','TEAM_ID','GAME_DATE'], as_index=False)['W'].sum()
games1['WINS_10GAMES'] = games1.groupby(['SEASON_ID','TEAM_ID'])['W'].apply(f).reset_index(drop=True, level=0)
games1['WINS_10GAMES'] = games1['WINS_10GAMES'].fillna(0) 

games = pd.merge(games,games1[['SEASON_ID','TEAM_ID','GAME_DATE','WINS_10GAMES']],on=['SEASON_ID','TEAM_ID','GAME_DATE'], how='left')



In [28]:
games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,PTS,DAYS_BTWN_GAMES,L,W,WINS_10GAMES
0,22016,1610612737,ATL,Atlanta Hawks,0021600014,2016-10-27,ATL vs. WAS,W,114,0.0,0,1,0.0
1,22016,1610612737,ATL,Atlanta Hawks,0021600026,2016-10-29,ATL @ PHI,W,104,2.0,0,1,1.0
2,22016,1610612737,ATL,Atlanta Hawks,0021600044,2016-10-31,ATL vs. SAC,W,106,2.0,0,1,2.0
3,22016,1610612737,ATL,Atlanta Hawks,0021600059,2016-11-02,ATL vs. LAL,L,116,2.0,1,0,3.0
4,22016,1610612737,ATL,Atlanta Hawks,0021600070,2016-11-04,ATL @ WAS,L,92,2.0,1,0,3.0


In [29]:
# Calculate the team's number of wins thus far in the season

#Sort by team & game date
games = games.sort_values(by=['SEASON_ID','TEAM_ID','GAME_DATE'], ascending=True)

# Calculate number of wins over season so far
f = lambda x: x.rolling(min_periods=1, window=82).sum().shift()

games1 = games.groupby(['SEASON_ID','TEAM_ID','GAME_DATE'], as_index=False)['W'].sum()
games1['WINS_UPTOGAME'] = games1.groupby(['SEASON_ID','TEAM_ID'])['W'].apply(f).reset_index(drop=True, level=0)
games1['WINS_UPTOGAME'] = games1['WINS_UPTOGAME'].fillna(0) 

games = pd.merge(games,games1[['SEASON_ID','TEAM_ID','GAME_DATE','WINS_UPTOGAME']],on=['SEASON_ID','TEAM_ID','GAME_DATE'], how='left')

#drop win/loss dummy
games = games.drop(columns=['W','L'])

games.head(50)


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,PTS,DAYS_BTWN_GAMES,WINS_10GAMES,WINS_UPTOGAME
0,22016,1610612737,ATL,Atlanta Hawks,0021600014,2016-10-27,ATL vs. WAS,W,114,0.0,0.0,0.0
1,22016,1610612737,ATL,Atlanta Hawks,0021600026,2016-10-29,ATL @ PHI,W,104,2.0,1.0,1.0
2,22016,1610612737,ATL,Atlanta Hawks,0021600044,2016-10-31,ATL vs. SAC,W,106,2.0,2.0,2.0
3,22016,1610612737,ATL,Atlanta Hawks,0021600059,2016-11-02,ATL vs. LAL,L,116,2.0,3.0,3.0
4,22016,1610612737,ATL,Atlanta Hawks,0021600070,2016-11-04,ATL @ WAS,L,92,2.0,3.0,3.0
5,22016,1610612737,ATL,Atlanta Hawks,0021600084,2016-11-05,ATL vs. HOU,W,112,1.0,3.0,3.0
6,22016,1610612737,ATL,Atlanta Hawks,0021600100,2016-11-08,ATL @ CLE,W,110,3.0,4.0,4.0
7,22016,1610612737,ATL,Atlanta Hawks,0021600111,2016-11-09,ATL vs. CHI,W,115,1.0,5.0,5.0
8,22016,1610612737,ATL,Atlanta Hawks,0021600132,2016-11-12,ATL vs. PHI,W,117,3.0,6.0,6.0
9,22016,1610612737,ATL,Atlanta Hawks,0021600154,2016-11-15,ATL @ MIA,W,93,3.0,7.0,7.0


In [30]:
#Calculate average points per team over last j days (within the current season)
#Note: if there are fewer than j days, just give the average points in 

#Sort by team & game date
games = games.sort_values(by=['SEASON_ID','TEAM_ID','GAME_DATE'], ascending=True)

# Calculate average points over last two games
for j in range(2,5,1):
    f = lambda x: x.rolling(min_periods=1, window=j).mean().shift()

    games1 = games.groupby(['SEASON_ID','TEAM_ID','GAME_DATE'], as_index=False)['PTS'].mean()
    games1['AVGPOINTS_'+str(j)+'GAMES'] = games1.groupby(['SEASON_ID','TEAM_ID'])['PTS'].apply(f).reset_index(drop=True, level=0)
    games = pd.merge(games,games1[['SEASON_ID','TEAM_ID','GAME_DATE','AVGPOINTS_'+str(j)+'GAMES']],on=['SEASON_ID','TEAM_ID','GAME_DATE'], how='left')

games.head()


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,PTS,DAYS_BTWN_GAMES,WINS_10GAMES,WINS_UPTOGAME,AVGPOINTS_2GAMES,AVGPOINTS_3GAMES,AVGPOINTS_4GAMES
0,22016,1610612737,ATL,Atlanta Hawks,0021600014,2016-10-27,ATL vs. WAS,W,114,0.0,0.0,0.0,NaN,NaN,NaN
1,22016,1610612737,ATL,Atlanta Hawks,0021600026,2016-10-29,ATL @ PHI,W,104,2.0,1.0,1.0,114.0,114.000000,114.0
2,22016,1610612737,ATL,Atlanta Hawks,0021600044,2016-10-31,ATL vs. SAC,W,106,2.0,2.0,2.0,109.0,109.000000,109.0
3,22016,1610612737,ATL,Atlanta Hawks,0021600059,2016-11-02,ATL vs. LAL,L,116,2.0,3.0,3.0,105.0,108.000000,108.0
4,22016,1610612737,ATL,Atlanta Hawks,0021600070,2016-11-04,ATL @ WAS,L,92,2.0,3.0,3.0,111.0,108.666667,110.0


In [31]:
#Combine two rows into one so unit of observation is game
gamelvl_df = pd.merge(games, games, on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
gamelvl_df = gamelvl_df.loc[~(gamelvl_df['TEAM_ID_x']==gamelvl_df['TEAM_ID_y'])]

#Keep away team as team "x" and home team as team "y"
gamelvl_df = gamelvl_df[gamelvl_df.MATCHUP_x.str.contains(' @ ')]

In [32]:
gamelvl_df.head()

,SEASON_ID,TEAM_ID_x,TEAM_ABBREVIATION_x,TEAM_NAME_x,GAME_ID,GAME_DATE,MATCHUP_x,WL_x,PTS_x,DAYS_BTWN_GAMES_x,...,TEAM_NAME_y,MATCHUP_y,WL_y,PTS_y,DAYS_BTWN_GAMES_y,WINS_10GAMES_y,WINS_UPTOGAME_y,AVGPOINTS_2GAMES_y,AVGPOINTS_3GAMES_y,AVGPOINTS_4GAMES_y
2,22016,1610612764,WAS,Washington Wizards,0021600014,2016-10-27,WAS @ ATL,L,99,0.0,...,Atlanta Hawks,ATL vs. WAS,W,114,0.0,0.0,0.0,NaN,NaN,NaN
5,22016,1610612737,ATL,Atlanta Hawks,0021600026,2016-10-29,ATL @ PHI,W,104,2.0,...,Philadelphia 76ers,PHI vs. ATL,L,72,3.0,0.0,0.0,97.0,97.000000,97.000000
10,22016,1610612758,SAC,Sacramento Kings,0021600044,2016-10-31,SAC @ ATL,L,95,2.0,...,Atlanta Hawks,ATL vs. SAC,W,106,2.0,2.0,2.0,109.0,109.000000,109.000000
14,22016,1610612747,LAL,Los Angeles Lakers,0021600059,2016-11-02,LAL @ ATL,W,123,1.0,...,Atlanta Hawks,ATL vs. LAL,L,116,2.0,3.0,3.0,105.0,108.000000,108.000000
17,22016,1610612737,ATL,Atlanta Hawks,0021600070,2016-11-04,ATL @ WAS,L,92,2.0,...,Washington Wizards,WAS vs. ATL,W,95,2.0,0.0,0.0,103.0,101.666667,101.666667


In [33]:
#Create combined points variable
gamelvl_df.loc[:,'TOT_PTS'] = gamelvl_df['PTS_x'] + gamelvl_df['PTS_y']

#Sort by team-date
gamelvl_df = gamelvl_df.sort_values(by=['TEAM_ID_x','GAME_DATE'], ascending=True)

In [34]:
gamelvl_df.head()

,SEASON_ID,TEAM_ID_x,TEAM_ABBREVIATION_x,TEAM_NAME_x,GAME_ID,GAME_DATE,MATCHUP_x,WL_x,PTS_x,DAYS_BTWN_GAMES_x,...,MATCHUP_y,WL_y,PTS_y,DAYS_BTWN_GAMES_y,WINS_10GAMES_y,WINS_UPTOGAME_y,AVGPOINTS_2GAMES_y,AVGPOINTS_3GAMES_y,AVGPOINTS_4GAMES_y,TOT_PTS
5,22016,1610612737,ATL,Atlanta Hawks,0021600026,2016-10-29,ATL @ PHI,W,104,2.0,...,PHI vs. ATL,L,72,3.0,0.0,0.0,97.0,97.000000,97.000000,176
17,22016,1610612737,ATL,Atlanta Hawks,0021600070,2016-11-04,ATL @ WAS,L,92,2.0,...,WAS vs. ATL,W,95,2.0,0.0,0.0,103.0,101.666667,101.666667,187
25,22016,1610612737,ATL,Atlanta Hawks,0021600100,2016-11-08,ATL @ CLE,W,110,3.0,...,CLE vs. ATL,L,106,3.0,6.0,6.0,115.0,119.333333,115.750000,216
37,22016,1610612737,ATL,Atlanta Hawks,0021600154,2016-11-15,ATL @ MIA,W,93,3.0,...,MIA vs. ATL,L,90,1.0,2.0,2.0,90.5,92.000000,90.250000,183
45,22016,1610612737,ATL,Atlanta Hawks,0021600174,2016-11-18,ATL @ CHA,L,96,2.0,...,CHA vs. ATL,W,100,3.0,7.0,7.0,104.0,106.333333,105.750000,196


In [35]:
#Import and merge distance measure
distances = pd.read_csv("distance_between_cities.csv", keep_default_na=False, na_values=[""])

gamelvl_df = pd.merge(gamelvl_df, distances, on=['TEAM_ABBREVIATION_x', 'TEAM_ABBREVIATION_y'])
gamelvl_df.head(50)

,SEASON_ID,TEAM_ID_x,TEAM_ABBREVIATION_x,TEAM_NAME_x,GAME_ID,GAME_DATE,MATCHUP_x,WL_x,PTS_x,DAYS_BTWN_GAMES_x,...,DAYS_BTWN_GAMES_y,WINS_10GAMES_y,WINS_UPTOGAME_y,AVGPOINTS_2GAMES_y,AVGPOINTS_3GAMES_y,AVGPOINTS_4GAMES_y,TOT_PTS,origin,destination,distance_miles
0,22016,1610612737,ATL,Atlanta Hawks,0021600026,2016-10-29,ATL @ PHI,W,104,2.0,...,3.0,0.0,0.0,97.0,97.000000,97.000000,176,Atlanta Hawks,Philadelphia,775.82
1,22016,1610612737,ATL,Atlanta Hawks,0021601111,2017-03-29,ATL @ PHI,W,99,1.0,...,1.0,5.0,28.0,100.0,105.666667,103.500000,191,Atlanta Hawks,Philadelphia,775.82
2,22017,1610612737,ATL,Atlanta Hawks,0021700107,2017-11-01,ATL @ PHI,L,109,3.0,...,2.0,3.0,3.0,113.5,110.333333,107.000000,228,Atlanta Hawks,Philadelphia,775.82
3,22018,1610612737,ATL,Atlanta Hawks,0021800088,2018-10-29,ATL @ PHI,L,92,2.0,...,2.0,3.0,3.0,106.5,115.000000,115.250000,205,Atlanta Hawks,Philadelphia,775.82
4,22018,1610612737,ATL,Atlanta Hawks,0021800621,2019-01-11,ATL @ PHI,W,123,2.0,...,2.0,7.0,27.0,119.0,114.666667,119.000000,244,Atlanta Hawks,Philadelphia,775.82
5,22016,1610612737,ATL,Atlanta Hawks,0021600070,2016-11-04,ATL @ WAS,L,92,2.0,...,2.0,0.0,0.0,103.0,101.666667,101.666667,187,Atlanta Hawks,Washington DC,636.35
6,22016,1610612737,ATL,Atlanta Hawks,0021601063,2017-03-22,ATL @ WAS,L,100,2.0,...,2.0,6.0,42.0,97.5,102.333333,103.500000,204,Atlanta Hawks,Washington DC,636.35
7,22017,1610612737,ATL,Atlanta Hawks,0021700177,2017-11-11,ATL @ WAS,L,94,1.0,...,2.0,5.0,6.0,105.0,105.666667,109.750000,207,Atlanta Hawks,Washington DC,636.35
8,22017,1610612737,ATL,Atlanta Hawks,0021701184,2018-04-06,ATL @ WAS,W,103,2.0,...,1.0,3.0,42.0,109.5,104.333333,105.000000,200,Atlanta Hawks,Washington DC,636.35
9,22018,1610612737,ATL,Atlanta Hawks,0021800556,2019-01-02,ATL @ WAS,L,98,2.0,...,4.0,3.0,14.0,111.0,105.666667,101.500000,212,Atlanta Hawks,Washington DC,636.35


In [36]:
# Import & Merge Rivalry info
rivalries = pd.read_csv("rivalries.csv", keep_default_na=True, na_values=[""])
gamelvl_df = pd.merge(gamelvl_df, rivalries, how="outer", on=['TEAM_ABBREVIATION_x', 'TEAM_ABBREVIATION_y']).fillna(0)

In [37]:
gamelvl_df = gamelvl_df.drop(columns=['origin','destination','Team A','Team B'])

gamelvl_df.head()

,SEASON_ID,TEAM_ID_x,TEAM_ABBREVIATION_x,TEAM_NAME_x,GAME_ID,GAME_DATE,MATCHUP_x,WL_x,PTS_x,DAYS_BTWN_GAMES_x,...,PTS_y,DAYS_BTWN_GAMES_y,WINS_10GAMES_y,WINS_UPTOGAME_y,AVGPOINTS_2GAMES_y,AVGPOINTS_3GAMES_y,AVGPOINTS_4GAMES_y,TOT_PTS,distance_miles,rivalry
0,22016,1610612737,ATL,Atlanta Hawks,0021600026,2016-10-29,ATL @ PHI,W,104,2.0,...,72,3.0,0.0,0.0,97.0,97.000000,97.00,176,775.82,0.0
1,22016,1610612737,ATL,Atlanta Hawks,0021601111,2017-03-29,ATL @ PHI,W,99,1.0,...,92,1.0,5.0,28.0,100.0,105.666667,103.50,191,775.82,0.0
2,22017,1610612737,ATL,Atlanta Hawks,0021700107,2017-11-01,ATL @ PHI,L,109,3.0,...,119,2.0,3.0,3.0,113.5,110.333333,107.00,228,775.82,0.0
3,22018,1610612737,ATL,Atlanta Hawks,0021800088,2018-10-29,ATL @ PHI,L,92,2.0,...,113,2.0,3.0,3.0,106.5,115.000000,115.25,205,775.82,0.0
4,22018,1610612737,ATL,Atlanta Hawks,0021800621,2019-01-11,ATL @ PHI,W,123,2.0,...,121,2.0,7.0,27.0,119.0,114.666667,119.00,244,775.82,0.0


In [44]:
#Add combined points of previous matchup
gamelvl_df['pre_PTS'] = gamelvl_df.groupby('MATCHUP_x')['TOT_PTS'].shift(1)

gamelvl_df.head(20)

,SEASON_ID,TEAM_ID_x,TEAM_ABBREVIATION_x,TEAM_NAME_x,GAME_ID,GAME_DATE,MATCHUP_x,WL_x,PTS_x,DAYS_BTWN_GAMES_x,...,DAYS_BTWN_GAMES_y,WINS_10GAMES_y,WINS_UPTOGAME_y,AVGPOINTS_2GAMES_y,AVGPOINTS_3GAMES_y,AVGPOINTS_4GAMES_y,TOT_PTS,distance_miles,rivalry,pre_PTS
0,22016,1610612737,ATL,Atlanta Hawks,0021600026,2016-10-29,ATL @ PHI,W,104,2.0,...,3.0,0.0,0.0,97.0,97.000000,97.000000,176,775.82,0.0,NaN
1,22016,1610612737,ATL,Atlanta Hawks,0021601111,2017-03-29,ATL @ PHI,W,99,1.0,...,1.0,5.0,28.0,100.0,105.666667,103.500000,191,775.82,0.0,176.0
2,22017,1610612737,ATL,Atlanta Hawks,0021700107,2017-11-01,ATL @ PHI,L,109,3.0,...,2.0,3.0,3.0,113.5,110.333333,107.000000,228,775.82,0.0,191.0
3,22018,1610612737,ATL,Atlanta Hawks,0021800088,2018-10-29,ATL @ PHI,L,92,2.0,...,2.0,3.0,3.0,106.5,115.000000,115.250000,205,775.82,0.0,228.0
4,22018,1610612737,ATL,Atlanta Hawks,0021800621,2019-01-11,ATL @ PHI,W,123,2.0,...,2.0,7.0,27.0,119.0,114.666667,119.000000,244,775.82,0.0,205.0
5,22016,1610612737,ATL,Atlanta Hawks,0021600070,2016-11-04,ATL @ WAS,L,92,2.0,...,2.0,0.0,0.0,103.0,101.666667,101.666667,187,636.35,0.0,NaN
6,22016,1610612737,ATL,Atlanta Hawks,0021601063,2017-03-22,ATL @ WAS,L,100,2.0,...,2.0,6.0,42.0,97.5,102.333333,103.500000,204,636.35,0.0,187.0
7,22017,1610612737,ATL,Atlanta Hawks,0021700177,2017-11-11,ATL @ WAS,L,94,1.0,...,2.0,5.0,6.0,105.0,105.666667,109.750000,207,636.35,0.0,204.0
8,22017,1610612737,ATL,Atlanta Hawks,0021701184,2018-04-06,ATL @ WAS,W,103,2.0,...,1.0,3.0,42.0,109.5,104.333333,105.000000,200,636.35,0.0,207.0
9,22018,1610612737,ATL,Atlanta Hawks,0021800556,2019-01-02,ATL @ WAS,L,98,2.0,...,4.0,3.0,14.0,111.0,105.666667,101.500000,212,636.35,0.0,200.0


In [ ]:
#Getting referee information for each of the games & combining into one dataset
#NOTE: This code takes FOREVER to run.

for i in final_games_data.GAME_ID:
    temp = boxscoresummaryv2.BoxScoreSummaryV2(game_id=i)   
    games_ref_temp = temp.officials.get_data_frame()
    games_ref_temp['GAME_ID'] = i
    games_ref = games_ref.append(games_ref_temp)

In [110]:
games_ref.head(20)    

,OFFICIAL_ID,FIRST_NAME,LAST_NAME,JERSEY_NUM,GAME_ID
0,1199,Tom,Washington,49,0021801230
1,202027,Tre,Maddox,73,0021801230
2,1626300,Ray,Acosta,54,0021801230
0,1199,Tom,Washington,49,0021801230
1,202027,Tre,Maddox,73,0021801230
2,1626300,Ray,Acosta,54,0021801230
0,1195,Derrick,Stafford,9,0021600026
1,2004,Courtney,Kirkland,61,0021600026
2,201246,Kevin,Cutler,34,0021600026
0,1198,Scott,Wall,31,0021601111


In [111]:
#First game is duplicated - fix that
games_ref2 = games_ref.drop_duplicates()
games_ref2.head()

,OFFICIAL_ID,FIRST_NAME,LAST_NAME,JERSEY_NUM,GAME_ID
0,1199,Tom,Washington,49,0021801230
1,202027,Tre,Maddox,73,0021801230
2,1626300,Ray,Acosta,54,0021801230
0,1195,Derrick,Stafford,9,0021600026
1,2004,Courtney,Kirkland,61,0021600026


In [112]:
#Export referee information to a csv file so that we can directly pull from that
#Rather than having to re-run the above code every time

games_ref2.to_csv('referees.csv')

In [42]:
#Use if importing referee info rather than going through the prior commands
games_ref2 = pd.read_csv('referees.csv')
games_ref2 = games_ref2.drop(columns=['Unnamed: 0'])
games_ref2.head()

,OFFICIAL_ID,FIRST_NAME,LAST_NAME,JERSEY_NUM,GAME_ID
0,1199,Tom,Washington,49,21801230
1,202027,Tre,Maddox,73,21801230
2,1626300,Ray,Acosta,54,21801230
3,1195,Derrick,Stafford,9,21600026
4,2004,Courtney,Kirkland,61,21600026


In [46]:
#Merge referee data in to create final dataset
#Note that as it currently stands, each game has multiple (3) rows because there are multiple (3) referees per game

gamelvl_df['GAME_ID'] = gamelvl_df['GAME_ID'].astype(int)

final_nba_data = pd.merge(gamelvl_df, games_ref2, how="outer", on=['GAME_ID'])
final_nba_data = final_nba_data.sort_values(by=['SEASON_ID','GAME_DATE'], ascending=True)
final_nba_data.head()

,SEASON_ID,TEAM_ID_x,TEAM_ABBREVIATION_x,TEAM_NAME_x,GAME_ID,GAME_DATE,MATCHUP_x,WL_x,PTS_x,DAYS_BTWN_GAMES_x,...,AVGPOINTS_3GAMES_y,AVGPOINTS_4GAMES_y,TOT_PTS,distance_miles,rivalry,pre_PTS,OFFICIAL_ID,FIRST_NAME,LAST_NAME,JERSEY_NUM
5535,22016,1610612752,NYK,New York Knicks,21600001,2016-10-25,NYK @ CLE,L,88,0.0,...,0.0,0.0,205,462.99,0.0,NaN,1363,Bennie,Adams,47
5536,22016,1610612752,NYK,New York Knicks,21600001,2016-10-25,NYK @ CLE,L,88,0.0,...,0.0,0.0,205,462.99,0.0,NaN,1662,Bill,Kennedy,55
5537,22016,1610612752,NYK,New York Knicks,21600001,2016-10-25,NYK @ CLE,L,88,0.0,...,0.0,0.0,205,462.99,0.0,NaN,202026,Karl,Lane,77
8119,22016,1610612759,SAS,San Antonio Spurs,21600003,2016-10-25,SAS @ GSW,W,129,0.0,...,0.0,0.0,229,1721.11,0.0,NaN,1152,Dan,Crawford,43
8120,22016,1610612759,SAS,San Antonio Spurs,21600003,2016-10-25,SAS @ GSW,W,129,0.0,...,0.0,0.0,229,1721.11,0.0,NaN,2003,Pat,Fraher,26


In [47]:
#Export final dataset to csv so that a separate Jupyter notebook can be used
#for creating the model
final_nba_data.to_csv('final_nba_data.csv')